# Notebook for Fruit Image classification
classification of fruit using fastai library (docs.fast.ai), which runs on top of pytorch

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

TODO provare a aggiungere altre metriche

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate
from fastai.callbacks import CSVLogger
from pathlib import Path
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from collections import namedtuple

In [3]:
path_img = Path("dataset_classification")
batch_size = 64
np.random.seed(20) #random seed for reproduce results

In [4]:
path_img

PosixPath('dataset_classification')

create dataset object using a folder source folder formatted as `label1/* label2/* ..`. Do not use cross validation, split it 80% 20 % for training validation. Apply all fastai builtin transformations and normalize (still need to understand well what it does)

#### Dataset creation
Information available at the following link https://docs.fast.ai/vision.data.html#ImageDataBunch

In [5]:
data = ImageDataBunch.from_folder(path_img,ds_tfms=get_transforms(), size=224, bs=batch_size).normalize(imagenet_stats)

In [6]:
data.classes

['Albicocca',
 'Banana',
 'Carota',
 'Cetriolo',
 'Champignon',
 'Ciliegia',
 'Cipolla',
 'Kiwi',
 'Lime',
 'Mela',
 'Patata',
 'Peperoncino',
 'Pera',
 'Pesca',
 'Prugna rossa',
 'Zucchino',
 'pomodoro_ciliegino',
 'pomodoro_sanmarzano']

In [ ]:
#show some images from dataset
data.show_batch(rows=3, figsize=(8,8))

#### CNN Model
Create a model using renet34 pretrained on imagenet

In [7]:
learn = create_cnn(data, models.resnet34, metrics=[error_rate], callback_fns=[CSVLogger])

/opt/miniconda/envs/fastai/lib/python3.7/site-packages/fastai/vision/learner.py:105: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")


Learning rate selection

In [ ]:
learn.lr_find(end_lr=100)

In [ ]:
learn.recorder.plot()

Regarding under/overfitting problems, please refer to the following links:
1. https://stats.stackexchange.com/questions/187335/validation-error-less-than-training-error/187404#187404
2. https://link.springer.com/article/10.1007/s13244-018-0639-9

In [ ]:
epochs = 3
lr = 1e-3
learn.fit_one_cycle(epochs, lr) #make 4 learning epoch

epoch,train_loss,valid_loss,error_rate,time
0,2.688975,0.716167,0.052632,02:32
1,1.343752,0.059801,0.000000,02:36


after 3 learning epoch we have achieved a 0% error rate

In [ ]:
learn.recorder.plot_lr(show_moms=True)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(8,12))
ax[0].plot(list(range(epochs)),learn.recorder.val_losses, label='Validation loss')
ax[0].plot(list(range(epochs)),[learn.recorder.losses[i] for i in range(epochs+2,epochs*(epochs+3),epochs+3)], label='Training loss')
#ax[0].plot(list(range(epochs)),[learn.recorder.losses[i] for i in range(0,len(learn.recorder.losses),epochs-3)], label='Training loss')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].legend(loc='upper right')
ax[1].plot(list(range(epochs)),learn.recorder.metrics)
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Error rate')

In [ ]:
#learn.save('roberto_model-1', return_path=True)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(10,10), dpi=150)

In [ ]:
interp.most_confused(min_val=1)

#### Commento di Roberto
Dai uno sguardo a questo paragrafo https://docs.fast.ai/vision.learner.html#ClassificationInterpretation
La probability è molto importante per capire la capacità predittiva di un modello.
Per ogni singola immagine è possibile capire con quale probabilità questa è stata associata al label corretto.
Più è alto tale valore, più ovviamente il modello funziona bene.
Se guardi la prima immagine in alto a sx, la prugna rossa ha una probability del 57% di essere prugna rossa (non è proprio così eccelente).
Estrarre le metriche da ogni immagine e mediarle per classe, ci può aiutare a descrivere la bontà del modello per ogni classe di prodotto ed a migliorarlo di conseguenza (facendo per esempio il transfer learning su altre CNN).
Insomma, non ci dobbiamo fermare all'errore totale di predizione (o, viceversa, alla sola accuratezza media del modello, che corrisponde a 1 - error_rate)

In [ ]:
interp.plot_top_losses(9, figsize=(11,11))

In [7]:
test_path = Path("dataset_classification_final_test/")
learn.load("roberto_model-1")

Learner(data=ImageDataBunch;

Train: LabelList (1138 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
Lime,Lime,Lime,Lime,Lime
Path: dataset_classification;

Valid: LabelList (304 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
Lime,Lime,Lime,Lime,Lime
Path: dataset_classification;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [8]:
Prediction = namedtuple('Prediction', ['path', 'pred_idx', 'pred_class', 'probability', 'misclassified']) #add actual_idx
def test_model_class(folder):   
    result = []    
    files_list = [files for files in folder.iterdir() if files.suffix == '.png']
    for f in files_list:
        misclass = False
        img = open_image(f)
        pred_class,pred_idx,outputs = learn.predict(img)
        if not (str(pred_class) == folder.name):
            print(f"Error! predicted class: '{pred_class}' vs actual class: '{folder.name}'")
            # print(img) @Simone I think that printing path makes more sense
            print("\t",f)
            misclass = True
        result.append(Prediction(f, int(pred_idx), str(pred_class), outputs, misclass))
        
    return result

def test_model_all(root_path):
    defaults.device = torch.device('cpu') #set cpu for inference (@Simone I am not really sure if it is a good idea, I'll investigate)
    tot = 0
    wrong = 0
    actual_idx = 0 #@Simone which is the purpose
    results = {}
    folders_list = sorted([folders for folders in test_path.iterdir()])
    for folder in folders_list:        
        result = test_model_class(folder)
        tot += len(result)
        wrong += [pred.misclassified for pred in result].count(True)
        actual_idx += 1
        results[folder.name] = result
    print(f"error rate: {round(wrong/tot*100,2)}%")
    defaults.device = torch.device('cuda') #set gpu again for training
    return results

In [9]:
results = test_model_all(test_path)

Error! predicted class: 'Prugna rossa' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_21.png
Error! predicted class: 'Prugna rossa' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_87.png
Error! predicted class: 'Patata' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_79.png
Error! predicted class: 'Patata' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_71.png
Error! predicted class: 'Patata' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_54.png
error rate: 0.81%


In [26]:
def pred_analysis(class_name, result_class,verbose=True, picture=False):
    """ takes a dict which is a element in the results dict generated by test_model_class
    TODO write what it does"""
    counter = 0
    no_errors = True
    if any([pred.misclassified for pred in result_class]):
        # no classification error
        if verbose:
            print(f"Actual {class_name} - no classification errors")
            return 1

    for c,i in enumerate(result_class): 
        if i.misclassified == True:
            idx = i.pred_idx
            prob = round(float(i.probability[idx])*100, 2)
            print(f"Actual {class_name} recognised as {i.pred_class} ( image n: {c}, pred_idx: {idx}, P: {str(prob)}%)")
            if picture:
                display(Image(str(i.path), width=100))
        
   
       
                      
def pred_error(fclass, fresults, verbose = False, picture = False):
    if fclass == 'all':
        for k,v in fresults.items():
            pred_analysis(k, v, verbose, picture)       
    else:
        pred_analysis(fclass, fresults[fclass], verbose, picture)

In [30]:
pred_error('Pesca', results, True, True)

Actual Pesca - no classification errors


In [45]:
# the function extracts the average probability values for an actual class
def pred_plot(actual_class, fresults, plot = True):
    try:
        # acquire the list of classes
        classes = list(fresults.keys())
                      
        # acquire the number of samples
        samples = len(results[actual_class]['probability'])
        
        ave_probs = []
                      
        for c in classes:
             # exctract the pred_idx from each class
            idx = fresults[c]['actual_idx'][0]
            temp_probs = []
             
            # extract the probability associated to each class for every sample          
            for s in range(samples):
                temp_probs.append(fresults[actual_class]['probability'][s][idx])
           
            # compute the average probability on class at time
            ave_probs.append(np.mean(temp_probs)) 

        # barplot colors
        bar_colors = ['gray']*(len(classes)+1)
        bar_colors[fresults[actual_class]['actual_idx'][0]] = 'blue'
        miss_array = np.array(results[actual_class]['misclassified'])
        index_array = np.array(results[actual_class]['pred_idx'])
        miss_classes = index_array[miss_array]
        miss_classes = np.unique(miss_classes)
        for i in miss_classes:
            bar_colors[i] = 'red'

        # barplot show
        plt.ylim(0, 1)
        plt.ylabel('Average probability')
        plt.xticks(range(len(ave_probs)), classes, rotation = 'vertical')
        plt.bar(range(len(ave_probs)), ave_probs, color = bar_colors)

        blue_patch = mpatches.Patch(color='blue', label='Correctly classified')
        red_patch = mpatches.Patch(color='red', label='Misclassified')
        gray_patch = mpatches.Patch(color='gray', label='Correctly discarded')
        plt.legend(handles=[blue_patch, gray_patch, red_patch])
        
        # averaged probability
        return ave_probs
    
    except KeyError:
        print("Not available class")

In [ ]:
pred_error('all', results, True, True)

In [47]:
defaults.device = torch.device('cpu')
tot = 0
wrong = 0
results = {}

for folder in test_path.iterdir():
    results[folder.name] = {'pred_idx': [], 'pred_class': [], 'probability': [], 'misClassified': []}
    for f in folder.iterdir():
        try:
            missClass = False
            img = open_image(f)
            pred_class,pred_idx,outputs = learn.predict(img)
            tot += 1
            if not (str(pred_class) == folder.name): 
                wrong +=1
                print(f"pred class: {pred_class} vs actual {folder.name}")
                print(img)
                missClass = True
            results[folder.name]['pred_idx'].append(int(pred_idx))
            results[folder.name]['pred_class'].append(str(pred_class))
            results[folder.name]['probability'].append(outputs)
            results[folder.name]['misClassified'].append(missClass)
        except IsADirectoryError:
            continue
        
print(f"error rate: {round(wrong/tot*100,2)}%")   

pred class: Prugna rossa vs actual Pesca
Image (3, 575, 362)
pred class: Prugna rossa vs actual Pesca
Image (3, 632, 380)
pred class: Patata vs actual Pesca
Image (3, 585, 406)
pred class: Patata vs actual Pesca
Image (3, 426, 552)
pred class: Patata vs actual Pesca
Image (3, 481, 358)
error rate: 0.81%


The plot here below refers to the content of the tutorial available at the following link:
https://www.tensorflow.org/tutorials/keras/basic_classification

In [48]:
probs = pred_plot('Pesca', results)

KeyError: 'actual_idx'

In [ ]:
probs

In [ ]:
dir(learn)

In [ ]:
learn.data

In [ ]:
learn.show_results()

In [49]:
learn.validate()

[0.05689092, tensor(0.0066)]